In [ ]:
from ultralytics import YOLO
import glob
import cv2
import numpy as np
import imutils
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as T

In [ ]:
# Export the model

yolo_model = YOLO('C:/Users/debora.assis/Documents/YoloV8_Glaucoma/Train_data/runs/segment/train/weights/yolo.pt' ) 

Segmentação - contorno do disco óptico

In [ ]:
path = 'C:/Users/debora.assis/Documents/0_Glaucoma/Dataset/REFUGE/test/images/'

# Mapeamento de cores para cada classe
class_colors = [(255, 0, 0), (255, 255, 0)]  # Exemplo: vermelho para classe 0, verde para classe 1

for image_path in glob.glob(path + "/*"):
    results = list(yolo_model(image_path, conf=0.1))
    result = results[0]

    if len(result) > 0:
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_name = image_path.split('\\')[1][:-4]

        img2 = cv2.resize(img, (640, 640))

        to_pil = T.ToPILImage()
        mask_result = [to_pil(mask) for mask in result.masks.data]

        img_mask = np.zeros_like(img2)  # Imagem para desenhar as máscaras

        for i, mask in enumerate(mask_result):
            
            mask_array = np.array(mask)
            
            color = class_colors[i]  # Cor correspondente à classe atual
            img_mask[mask_array.astype(bool)] = color
            color = class_colors[i]  # Cor correspondente à classe atual
            contours = cv2.findContours(mask_array, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = imutils.grab_contours(contours)
            contours = max(contours, key=cv2.contourArea)
            cv2.drawContours(img2, [contours], -1, color, thickness=2)

        fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(10, 8))

        ax = ax.flatten()
        for a in ax:
            ax[0].imshow(img)
            ax[0].set_title(str(img_name), fontsize=12)
            ax[0].set_xticks([])
            ax[0].set_yticks([])

            ax[1].imshow(np.array(img_mask))
            ax[1].set_title('Predicted Mask', fontsize=12)
            ax[1].set_xticks([])
            ax[1].set_yticks([])

            ax[2].imshow(img2)
            ax[2].set_title('Contour Image', fontsize=12)
            ax[2].set_xticks([])
            ax[2].set_yticks([])

            plt.show()


Localização do disco óptico 

In [ ]:
def yolov8_detect(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    res = list(yolo_model(image_path, conf=0.1))
    # Loop através dos resultados da detecção
    max_confidence = -1  # Initialize with a negative value
    best_box = None

    for result in res:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            confidence = float(result.boxes.conf)
            label = int(result.boxes.cls)

            # Check if the current confidence is higher than the maximum confidence
            if confidence > max_confidence:
                max_confidence = confidence
                best_box = (x1, y1, x2, y2)

    # Now, best_box contains the coordinates of the bounding box with the highest confidence.
    if best_box is not None:
        x1, y1, x2, y2 = best_box
        color = (0, 255, 0)  # Cor da bounding box (verde)
        thickness = 2  # Espessura da linha
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

        # Adicionar rótulo e confiança
#         text = f'Class: {label}, Conf: {confidence:.2f}'
        text = f'Conf: {confidence:.2f}'
        if confidence > 0.1:
            i = 1
        else:
            i = 0
        cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, thickness)

    # Mostrar a imagem com as bounding boxes
    plt.imshow(image)
    plt.show()
    return i

In [ ]:
# Refuge teste
path = 'C:/Users/debora.assis/Documents/YoloV8_Glaucoma/Test_predict/data_test_refuge/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolov8_detect(img_path) 
    i.append(a)


In [ ]:
len(i)

In [ ]:
# DRISHT-GS
path = 'C:/Users/debora.assis/Documents/YoloV8_Glaucoma/Test_predict/teste_externo/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolov8_detect(img_path) 
    i.append(a)
    

In [ ]:
sum(i)

In [ ]:
# ORIGA
path = 'D:/Glaucoma/Banco_de_dados/ORIGA/glaucoma/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolov8_detect(img_path) 
    i.append(a)


In [ ]:
sum(i)  

In [ ]:
# ORIGA
path = 'D:/Glaucoma/Banco_de_dados/ORIGA/sanas/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolov8_detect(img_path) 
    i.append(a)


In [ ]:
sum(i)

In [ ]:
# HRF
path = 'D:/Glaucoma/Banco_de_dados/HRF/images/Glaucoma/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolov8_detect(img_path) 
    i.append(a)

In [ ]:
sum(i)  

In [ ]:
# HRF
path = 'D:/Glaucoma/Banco_de_dados/HRF/images/Normal/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolov8_detect(img_path) 
    i.append(a)

In [ ]:
sum(i)

In [ ]:
# HRF
path = 'D:/Glaucoma/Banco_de_dados/HRF/images/Retinopatia/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolov8_detect(img_path) 
    i.append(a)

In [ ]:
sum(i)